# 시맨틱 검색(Semantic search) quick start

<a target="_blank" href="https://colab.research.google.com/github/elastic/elasticsearch-labs/blob/main/notebooks/search/00-quick-start.ipynb"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

이 대화형 노트북은 공식 [Elasticsearch Python 클라이언트](https://www.elastic.co/guide/en/elasticsearch/client/python-api/current/connecting.html)를 사용하여 Elasticsearch의 몇 가지 기본 작업을 소개합니다.
텍스트 임베딩을 위해 [Sentence Transformers](https://www.sbert.net)를 사용하여 시맨틱 검색을 수행합니다. 하이브리드 검색 시스템을 위해 기존 텍스트 기반 검색을 시맨틱 검색과 통합하는 방법을 알아보세요.

## Elasticsearch/Kibana 실행

먼저 Elasticsearch/Kibana를 실행하세요.

## Install packages and import modules

시작하려면 Python 클라이언트를 사용하여 Elastic 배포에 연결해야 합니다.

먼저 `elasticsearch` Python 클라이언트를 설치해야 합니다.

In [ ]:
!pip install -qU elasticsearch sentence-transformers==2.2.2

# Setup the Embedding Model

이 예에서는 `sentence_transformers` 라이브러리의 일부인 `paraphrase-multilingual-MiniLM-L12-v2`를 사용하고 있습니다. 이 모델에 대한 자세한 내용은 [Huggingface](https://huggingface.co/sentence-transformers/paraphrase-multilingual-MiniLM-L12-v2)에서 읽어보실 수 있습니다.

In [ ]:
import os
cwd = os.getcwd()

try :
    os.mkdir(cwd + "/models") 
except:
    print('해당 경로가 이미 존재합니다.')

In [ ]:
os.chdir(cwd + "/models")

try :
    os.system("git clone https://huggingface.co/sentence-transformers/paraphrase-multilingual-MiniLM-L12-v2")
except:
    print('해당 경로가 이미 존재합니다.')

os.chdir(cwd)

In [ ]:
from sentence_transformers import SentenceTransformer
import torch

device = 'cuda' if torch.cuda.is_available() else 'cpu'

model = SentenceTransformer(cwd + '/models/paraphrase-multilingual-MiniLM-L12-v2', device=device)
model

## Elasticsearch 클라이언트 초기화

이제 [Elasticsearch Python 클라이언트](https://www.elastic.co/guide/en/elasticsearch/client/python-api/current/index.html) 인스턴스를 생성합니다.   
Eleasticsearch URL, 사용자 ID, 사용자 Password, 인증서 경로 정보가 필요하며, 인증서 경로는 (auto_install 시) `kibana.yml` 하단에 기재되어 있습니다.
Elasticsearch에 로컬로 연결하는 방법에 대해  [자세히 알아보세요](https://www.elastic.co/guide/en/elasticsearch/client/python-api/current/connecting.html#_verifying_https_with_certificate_fingerprints_python_3_10_or_later)

In [ ]:
from elasticsearch import Elasticsearch
from getpass import getpass

ES_URL = getpass('Elasticsearch URL: ')
ES_ID = "elastic" 
ES_PW = getpass('elastic user PW: ')
CERT_PATH = getpass('Elasticsearch cer 파일 경로: ')

# Create the client instance
client = Elasticsearch(
    ES_URL,
    basic_auth=(ES_ID, ES_PW),
    ca_certs=CERT_PATH
)

클라이언트가 이 테스트에 연결되었는지 확인합니다.

In [ ]:
print(client.info())

In [ ]:
if client.indices.exists(index="book_index"):
    client.indices.delete(index="book_index")

## Index some test data

클라이언트가 설정되어 Elastic 배포에 연결되었습니다.
이제 Elasticsearch 쿼리의 기본 사항을 테스트하려면 일부 데이터가 필요합니다.
다음 필드가 포함된 작은 도서 색인을 사용하겠습니다.

- `title`
- `authors`
- `publish_date`
- `num_reviews`
- `publisher`

### Create index

테스트 데이터에 대한 올바른 매핑을 사용하여 Elasticsearch 인덱스를 생성해 보겠습니다.

In [ ]:
# Create the index
client.indices.create(index='book_index', mappings = {
    "properties": {
        "title_vector": {
            "type": "dense_vector",
            "dims": 384,
            "index": "true",
            "similarity": "cosine"
        }
    }
})


### Index test data

다음 명령을 실행하여 이 [데이터 세트](https://raw.githubusercontent.com/elastic/elasticsearch-labs/main/notebooks/search/data.json)의 인기 프로그래밍 서적 10권에 대한 정보가 포함된 일부 테스트 데이터를 업로드하십시오. ).
`model.encode`는 이전에 초기화한 모델을 사용하여 텍스트를 즉시 벡터로 인코딩합니다.

In [ ]:
import json
from urllib.request import urlopen
import os

cwd = os.getcwd()
url = cwd + "/data.json"
response = open(url)
books = json.load(response)

actions = []
for book in books:
    actions.append({"index": {"_index": "book_index"}})
    # Transforming the title into an embedding using the model
    book["title_vector"] = model.encode(book["title"]).tolist()
    actions.append(book)
client.bulk(index="book_index", operations=actions)


## 참고: Elasticsearch 응답을 예쁘게 인쇄합니다.

API 호출은 읽기 어려운 중첩 JSON을 반환합니다.
예제에서 사람이 읽을 수 있는 멋진 출력을 반환하기 위해 `pretty_response`라는 작은 함수를 만들겠습니다.

In [ ]:
def pretty_response(response):
    for hit in response['hits']['hits']:
        id = hit['_id']
        publication_date = hit['_source']['publish_date']
        score = hit['_score']
        title = hit['_source']['title']
        summary = hit['_source']['summary']
        publisher = hit["_source"]["publisher"]
        num_reviews = hit["_source"]["num_reviews"]
        authors = hit["_source"]["authors"]
        pretty_output = (f"\nID: {id}\nPublication date: {publication_date}\nTitle: {title}\nSummary: {summary}\nPublisher: {publisher}\nReviews: {num_reviews}\nAuthors: {authors}\nScore: {score}")
        print(pretty_output)

## Making queries

이제 책의 색인을 생성했으므로 주어진 쿼리와 유사한 책에 대한 시맨틱 검색을 수행하려고 합니다.
쿼리를 삽입하고 검색을 수행합니다.

In [ ]:
response = client.search(index="book_index",
    knn= {
      "field": "title_vector",
      "query_vector": model.encode("가장 좋은 실용주의 프로그래밍 책"),
      "k": 3,
      "num_candidates": 100
    }
)

pretty_response(response)

## Filtering

필터 컨텍스트는 주로 구조화된 데이터를 필터링하는 데 사용됩니다. 예를 들어 필터 컨텍스트를 사용하여 다음과 같은 질문에 답할 수 있습니다.

- _이 타임스탬프가 2015년에서 2016년 범위에 속합니까?_
- _상태 필드가 '게시됨'으로 설정되어 있나요?_

필터 컨텍스트는 'bool' 쿼리의 'filter' 또는 'must_not' 매개변수와 같은 필터 매개변수에 쿼리 절이 전달될 때마다 적용됩니다.

Elasticsearch 문서의 필터 컨텍스트에 대해 [자세히 알아보세요](https://www.elastic.co/guide/en/elasticsearch/reference/current/query-filter-context.html#filter-context).

### Example: Keyword Filtering

쿼리에 키워드 필터를 추가하는 예시입니다.

"publisher" 필드가 "addison"와 동일한 문서만 포함하여 결과의 ​​범위를 좁힙니다.

코드는 "가장 좋은 실용주의 프로그래밍 책?"와 유사한 인기 도서를 검색합니다. 제목 벡터를 기반으로 하며 게시자로 "addison"를 지정합니다.

In [ ]:
response = client.search(index="book_index", knn= {
    "field": "title_vector",
    "query_vector": model.encode("가장 좋은 실용주의 프로그래밍 책?"),
    "k": 3,
    "num_candidates": 100,
    "filter": {
        "bool": {
            "should": [{
                "term": {
                    "publisher": "addison"
                }
            }]
        }
    }
})

pretty_response(response)

### Example: Advanced Filtering

Elasticsearch의 고급 필터링을 사용하면 조건을 적용하여 정확한 검색 결과를 구체화할 수 있습니다.
다양한 연산자를 지원하며 특정 필드, 범위 또는 조건을 기반으로 결과를 필터링하는 데 사용할 수 있어 검색 결과의 정확성과 관련성을 높일 수 있습니다.
이 [쿼리 및 필터 컨텍스트 예시](https://www.elastic.co/guide/en/elasticsearch/reference/current/query-filter-context.html#query-filter-context-ex)에서 자세히 알아보세요.

In [ ]:
response = client.search(index="book_index", knn = {
    "field": "title_vector",
    "query_vector": model.encode("가장 좋은 실용주의 프로그래밍 책?"),
    "k": 10,
    "num_candidates": 100,
    "filter": {
        "bool": {
            "should": [
                {
                  "term": {
                      "publisher": "addison"
                  }
                },
                {
                  "term": {
                      "authors": "david"
                  }
                }
            ],

        }
    }
  }
)

pretty_response(response)

## Hybrid Search

이 사례에서는 텍스트 검색을 위한 BM25와 최근접 이웃 검색을 위한 HNSW라는 두 가지 검색 알고리즘의 조합을 조사합니다. BM25와 조밀한 벡터 임베딩을 생성하는 ML 모델과 같은 여러 순위 방법을 결합하면 최상의 순위 결과를 얻을 수 있습니다. 이 접근 방식을 통해 각 알고리즘의 장점을 활용하고 전반적인 검색 성능을 향상시킬 수 있습니다.

[Reciprocal Rank Fusion(RRF)](https://www.elastic.co/guide/en/elasticsearch/reference/current/rrf.html)은 다양한 정보 검색 전략 결과를 결합하기 위한 최첨단 순위 알고리즘입니다.
RRF는 보정 없이도 다른 모든 순위 알고리즘보다 성능이 뛰어납니다.
간단히 말해서, 즉시 사용 가능한 최고의 하이브리드 검색이 가능합니다.

In [ ]:
response = client.search(index="book_index",
    query= {
        "match": {
            "summary": "python"
        },
    },
    knn = {
        "field": "title_vector",
        # generate embedding for query so it can be compared to `title_vector`
        "query_vector" : model.encode("python 프로그래밍 방법론").tolist(),
        "k": 5,
        "num_candidates": 10
    },
    rank = {
        "rrf": {
            "window_size": 100,
            "rank_constant": 20
        }
    }
)

pretty_response(response)